# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Y. Wu  ->  Y. Wu  |  ['Y. Wu']
S. Kumar  ->  S. Kumar  |  ['S. Kumar']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Y. Wu  ->  Y. Wu  |  ['Y. Wu']
Arxiv has 55 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2601.00127
extracting tarball to tmp_2601.00127...

 done.
Retrieving document from  https://arxiv.org/e-print/2601.00415
extracting tarball to tmp_2601.00415...

 done.
Retrieving document from  https://arxiv.org/e-print/2601.00439


extracting tarball to tmp_2601.00439...

 done.
Retrieving document from  https://arxiv.org/e-print/2601.00636
extracting tarball to tmp_2601.00636...

 done.
Retrieving document from  https://arxiv.org/e-print/2601.00640
extracting tarball to tmp_2601.00640...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.00415-b31b1b.svg)](https://arxiv.org/abs/2601.00415) | **The Double-Peaked Calcium-Strong SN 2025coe: Progenitor Constraints from Early Interaction and Ejecta Asymmetries**  |
|| A. P. Ravi, et al. -- incl., <mark>S. Kumar</mark> |
|*Appeared on*| *2026-01-05*|
|*Comments*| *28 pages, 17 figures, 3 tables; Submitted to ApJ*|
|**Abstract**|            Supernova (SN) 2025coe at a distance of $\sim$25 Mpc is the third-closest calcium-strong (CaST) transient. It was discovered at a large projected offset of $\sim$34 kpc from its potential host galaxy NGC 3277. Multiband photometry of SN 2025coe indicates the presence of two peaks at day $\sim$2 and day $\sim$11 after explosion. Modeling the bolometric light curve, we find that the first peak can be reproduced either by shock cooling of a compact envelope ($R_\mathrm{env}$ $\approx $6-40 $R_{\odot}$; $M_\mathrm{env}$ $\approx $0.1-0.2 $M_{\odot}$) or by interaction with close-in circumstellar material (CSM; $R_{\mathrm{CSM}} \lesssim 8 \times10^{14}$ cm), or a combination of both. The second peak is dominated by radioactive decay of $^{56}$Ni ($M_{\mathrm{ej}} \approx $0.4-0.5 $M_{\odot}$; $M_{^{56}\mathrm{Ni}} \approx 1.4 \times 10^{-2}$ $M_{\odot}$). SN 2025coe rapidly evolves from the photospheric phase dominated by He I P-Cygni profiles to nebular phase spectra dominated by strong [Ca II] $\lambda \lambda$7291, 7323 and weak [O I] $\lambda \lambda$6300, 6364 emission lines. Simultaneous line profile modeling of [Ca II] and [O I] at nebular phases shows that an asymmetric core-collapse explosion of a low-mass ($\lesssim$3.3 $M_{\odot}$) He-core progenitor can explain the observed line profiles. Alternatively, lack of local star formation at the site of the SN explosion combined with a low ejecta mass is also consistent with a thermonuclear explosion due to a low-mass hybrid He-C/O white dwarf + C/O white dwarf merger.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.00439-b31b1b.svg)](https://arxiv.org/abs/2601.00439) | **A global view of post-interaction white dwarf-main sequence binaries**  |
|| C. Shariat, <mark>K. El-Badry</mark> |
|*Appeared on*| *2026-01-05*|
|*Comments*| *21 pages, 8 figures, plus appendices. Submitted to PASP, comments are welcome*|
|**Abstract**|            Common-envelope evolution (CEE) is among the most uncertain phases in binary evolution. To empirically constrain CEE, we construct a uniformly selected sample of eclipsing post--common-envelope binaries (PCEBs). Starting from an unresolved white dwarf-main-sequence (WDMS) candidate sample within 200 pc selected from the Gaia color-magnitude diagram, we identify 39 detached eclipsing WDMS binaries using ZTF light curves. The binaries contain cool M dwarfs orbiting warm white dwarfs with orbital periods ($P_{\rm orb}$) of 0.1-2 d. The sample's simple selection function allows us to model observational incompleteness and infer intrinsic properties of the PCEB population. We find an orbital-period distribution consistent with being log-uniform over 0.1-2 d, contrary to recent reports of a bimodal distribution. The companion-mass distribution peaks around $0.25~{\rm M_\odot}$ and declines steeply toward larger masses. The estimated local space density is $7.2\times10^{-5}~{\rm pc^{-3}}$, corresponding to a Galaxy-wide birth rate of 0.01 per year. Combining our results with recent Gaia-based constraints on wider WDMS binaries, we construct an empirical period distribution of post-interaction WDMS binaries spanning 0.1-1000 d. The emerging period distribution is roughly log-flat (d$N/{\rm d}\log P_{\rm orb}\propto P_{\rm orb}^0$) at $P_{\rm orb} < 2$ d and log-increasing (d$N/{\rm d}\log P_{\rm orb}\propto P_{\rm orb}^1$) at $P_{\rm orb} = 100-1000$ d. The 10-100 d regime remains poorly constrained, but a few nearby systems suggest it is also well-populated. Short-period PCEBs ($P_{\rm orb}<2$ d) with M dwarf companions are roughly 2-3 times more common than wide ($P_{\rm orb} = 100-1000$ d) WDMS binaries with FGK companions, which likely formed through stable mass transfer. These results provide direct observational constraints on CEE and an empirical benchmark for binary-population models.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.00636-b31b1b.svg)](https://arxiv.org/abs/2601.00636) | **The SRG/eROSITA all-sky survey: The morphologies of clusters of galaxies: II. The intrinsic distributions of morphological parameters**  |
|| J. S. Sanders, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2026-01-05*|
|*Comments*| *19 pages, 10 figures, accepted by A&A*|
|**Abstract**|            X-ray selected surveys of clusters of galaxies have been reported to contain more regular cool core clusters compared to samples selected using the Sunyaev-Zel'dovich (SZ) effect. Morphology population studies on X-ray selected clusters will be biased without taking into account selection, as cool cores are more easily detected at low redshifts, but can be mistaken for point sources at high redshift. eROSITA, aboard SRG, found over 12000 optically-identified clusters in its first survey, eRASS1. Taking account of the selection function obtained from simulations, we obtain using a Bayesian framework the intrinsic distribution of morphological parameters, including the concentration, central density, cuspiness, ellipticity and slosh. We construct scaling relations for the parameters as a function of redshift (z) and luminosity (LX), and study their distribution within z or LX bins. We find that the concentration in a scaled aperture evolves positively with LX, similarly to the central scaled density, and negatively with z. When using a fixed aperture, its evolution with LX is lower, but also dependent on the choice of cluster centre. The mean ellipticity does not significantly evolve with z or LX. eRASS1 clusters show indications of higher concentrations compared to SZ-selected objects, even after taking account the selection; this suggests that if our X-ray selection model is correct SZ-selected clusters may also suffer from morphological selection effects. We compare different parameter distribution models in bins of z and LX. The distribution of concentration and ellipticity is generally consistent with a normal one, but other parameters such as the central density and cuspiness strongly favour more complex distributions. However, modelling of all clusters as a single population generally prefers non-normal distributions. [abridged]         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.00640-b31b1b.svg)](https://arxiv.org/abs/2601.00640) | **Evidence for a Nonzero Eccentricity Superpuff Exoplanet WASP-107 b Using JWST Occultation Observation**  |
|| <mark>Y. Wu</mark>, et al. |
|*Appeared on*| *2026-01-05*|
|*Comments*| *17 pages, 12 figures, accepted for publication in The ApJL*|
|**Abstract**|            WASP-107~b is an extremely low-density super-puff exoplanet whose inflated radius and evidence of strong internal heating make it a key target for understanding planetary structure and evolution. Its orbital eccentricity is a critical parameter for testing mechanisms such as tidal heating and high-eccentricity migration, yet previous measurements have remained inconclusive. Due to the large radial velocity jitter caused by stellar activity, and the presence of at least one additional planet in the system, previous radial velocity measurements could not robustly determine the eccentricity of WASP-107~b. Here we combine the new JWST secondary eclipse data with transit timing data from HST, TESS, and JWST to measure the eccentricity of WASP-107~b. Our joint analysis shows that WASP-107~b has an eccentricity of $0.09\pm0.02$, a mass of $0.096\pm0.005 \, M_J$, and an orbital period of $5.721487\pm0.000001$~days. We find the $99.7\%$ lower limit of the eccentricity is about 0.04. These new measurements are consistent with the scenario in which WASP-107~b is in the final stage of high-eccentricity migration. Preliminary estimate shows that eccentricity-driven tidal dissipation can provide a significant contribution to the energy required to sustain the observed radius inflation of WASP-107~b. Our results establish the dynamical status of one of the most intriguing low-density exoplanets known, and offer new insights into its formation and evolution history.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2601.00127-b31b1b.svg)](https://arxiv.org/abs/2601.00127) | **Light-tight skipper-CCDs for X-ray detection in space**  |
|| A. M. Botti, et al. -- incl., <mark>Y. Wu</mark> |
|*Appeared on*| *2026-01-05*|
|*Comments*| *9 pages, 11 figures*|
|**Abstract**|            Skipper Charge-Coupled Devices (skipper-CCDs) are pixelated silicon detectors with deep sub-electron resolution. Their radiation hardness and capability to reconstruct energy deposits with unprecedented precision make them a promising technology for space-based X-ray astronomy. In this scenario, optical and near-infrared photons may saturate the sensor, distorting the reconstructed signal. We present a light-tight shield for skipper-CCDs to suppress optical backgrounds while preserving X-ray detection efficiency. We deposited thin aluminum layers on the CCD surface using an e-beam evaporator and evaluated their blinding performance across wavelengths from 650 to 1000 nm using a monochromator, as well as the X-ray transmission using an $^{55}$Fe source. We find that 50 and 100\,nm layers provide >99.6\% light suppression, with no efficiency loss for 5.9 and 6.4\,keV X-rays. In addition, we used Geant4 simulations to extend these results to a broader energy range and quantify the efficiency loss for different aluminum thicknesses. Results show that thin aluminum coatings are an effective, low-cost solution for optical suppression in skipper-CCDs intended for X-ray detection and space instrumentation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

115  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

0  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
